In [1]:
import numpy as np
import cv2

import os
import glob
from multiprocessing import Pool

In [4]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image 
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [5]:
FOV = np.pi
DIM = (1280, 1280)

# camera rotation
hcam_deg = 0
vcam_deg = 0
xcam_deg = -45

hcam_rad = hcam_deg * np.pi / 180.0
vcam_rad = vcam_deg * np.pi / 180.0
xcam_rad = xcam_deg * np.pi / 180.0
rot_x = np.array([
    [1, 0, 0],
    [0, np.cos(xcam_rad), -np.sin(xcam_rad)],
    [0, np.sin(xcam_rad), np.cos(xcam_rad)]
])
rot_y = np.array([
    [np.cos(vcam_rad), 0, np.sin(vcam_rad)],
    [0, 1, 0],
    [-np.sin(vcam_rad), 0, np.cos(vcam_rad)]
])
rot_z = np.array([
    [np.cos(hcam_rad), -np.sin(hcam_rad), 0],
    [np.sin(hcam_rad), np.cos(hcam_rad), 0],
    [0, 0, 1]
])

mask = create_circular_mask(DIM[0], DIM[1], radius=DIM[0]/2-10)

In [4]:
def mapping(x, y):
    pfish = np.zeros((2, 1))
    psph = np.zeros((3, 1))
    width = DIM[0]
    height = DIM[1]
    theta = 2.0 * 3.14159265 * (x / 2560 - 0.5) # -pi to pi
    phi = 3.14159265 * (y / 1280 - 0.5)	# -pi/2 to pi/2

    psph[0] = np.cos(phi) * np.sin(theta)
    psph[1] = np.cos(phi) * np.cos(theta)
    psph[2] = np.sin(phi)

    psph = rot_z.dot(psph)
    psph = rot_x.dot(psph)

    theta = np.arctan2(psph[2], psph[0])
    phi = np.arctan2(np.linalg.norm(psph[0:3:2]), psph[1]) 
    r = width / FOV * 1.55 * np.tan(phi/2)
    # r = width / FOV * 2 * np.sin(phi/2)

    # Pixel in fisheye space
    pfish[0] = 0.5 * width + r * np.cos(theta)
    pfish[1] = 0.5 * width + r * np.sin(theta)

    return np.around(pfish).astype(int)

In [5]:
def filter(ind):
    i = ind // 2560
    j = ind % 2560
    fishcoord = mapping(j, i)
    fishcoord[fishcoord >= DIM[0]] = DIM[0] - 1
    fishcoord[fishcoord < 0] = 0
    return (fishcoord[1], fishcoord[0])

In [12]:
equirec_i = np.zeros((1280, 2560), int)
equirec_j = np.zeros((1280, 2560), int)
pool = Pool()
coords = pool.map(filter, range(1280*2560))
for ind in range(1280*2560):
    i = ind // 2560
    j = ind % 2560

    if (False):
        equirec_i[i, j] = 0
        equirec_j[i, j] = 0
    else:
        equirec_i[i, j] = coords[ind][0]
        equirec_j[i, j] = coords[ind][1]

In [13]:
np.save('equirec_i_0', equirec_i)
np.save('equirec_j_0', equirec_j)

In [2]:
equirec_i = np.load('/home/shane/stitching/image_processing/equirec_i_0.npy')
equirec_j = np.load('/home/shane/stitching/image_processing/equirec_j_0.npy')

In [7]:
img = cv2.imread('/home/shane/0.jpg')
img[~mask] = 0 
new_img = np.zeros((1280, 2560, 3), np.uint8)
new_img = img[equirec_i, equirec_j]
cv2.imwrite('/home/shane/0as.jpg', new_img) 
cv2.imshow('test', new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()